In [ ]:
#This utility can take multiple CSV output files, each with same format and produce the delta of changes between the 
#files. Each file is assumed to be a database comparison between two databases that are synced using some process or tool.
#It also takes care of data sync lag between the data sources by removing records that may not have yet have been confirmed 
#to be different (could be different because of a time lag).

In [ ]:
import numpy as np
import pandas as pd
import xlsxwriter
import re

In [ ]:
#default metrics dict
#add fields here...
metrics_base = {
        'f1_mismatch':0,
        'f2_mismatch':0
    }
    

In [ ]:
#cleanup of the dataframe is the first step - here we baseline values
default_values = {
    'f1_src1': 'UNSET', 'f1_src2': 'UNSET', 
    'f2_src1': 'NOT_COMPLETED', 'f2_src2': 'NOT_COMPLETED'
         }

In [ ]:
def dq_clean(dq_df):
    
    dq_df = dq_df.fillna(value=default_values)
    
    #add metrics columns inside the dataframe as well
    for metric, value in metrics_base.items():
        dq_df[metric] = value
    
    return dq_df

In [ ]:
def update_metrics(row):
    if row['f1_src1'] != row['f1_src2']:
        row['mismatch'] = row['f1_mismatch'] = True
    if row['f2_src1'] != row['f2_src2']:
        row['mismatch'] = row['f2_mismatch'] = True        
    return row

In [ ]:
def sum_metrics(dq_df):
    #initialize the metrics to 0 first
    metrics = dict.fromkeys(metrics_base, 0)
    metrics['f1_mismatch'] = dq_df.value_counts('f1_mismatch')[True] if True in dq_df.value_counts('f1_mismatch') else 0
    metrics['f2_mismatch'] = dq_df.value_counts('f2_mismatch')[True] if True in dq_df.value_counts('f2_mismatch') else 0

    return metrics

In [ ]:
def dq_analyze(dq_df, dq_date):
    dq_df = dq_df.apply(update_metrics, axis=1)
    
    #drop the rows that dont have mismatch that we want to consider
    dq_df = dq_df[dq_df['mismatch'] == True]
    
    return dq_df

In [ ]:
from collections import OrderedDict
dq_dfDict = OrderedDict()

def dq_df_add(dq_df, dq_date):
    
    dfB = dq_df.set_index('main_key')
    for date, dfA in dq_dfDict.items():
        
        #step 1 is to provide only the increments over the previously found users, so if
        #B is the new set, we need to do a B-A1-A2-A3 etc.
  
        #dfA = dfA.set_index('x_user_id') -- not needed since all previous dataframes in dict are having this index
        dfB = dfB.loc[dfB.index.difference(dfA.index), ]
        
        #second order of business is to now do a A1-(A1-B), A2-(A2-B), A3-(A3-B) and rewrite all previous dataframes 
        #to remove any records we thought were previously not in sync but because they no longer appeared in B, that
        #means that they are in sync now, so we need to remove them from A
        dfB1 = dq_df.set_index('main_key')
        dfX = dfA.loc[dfA.index.difference(dfB1.index), ]
        dfA = dfA.loc[dfA.index.difference(dfX.index), ]
        #dfA.to_excel(writer, sheet_name=date) - writing will be done all at once
        #instead we need to change the item in dict
        dq_dfDict[date] = dfA

    #At the end of the loop, now we can write B down
    #this also handles the case of the first element insertion where we do not enter the loop at all
    dq_dfDict[dq_date] = dfB
    #dfB.to_excel(writer, sheet_name=dq_date) -- no need to save

In [ ]:
#define a special date based metrics dictionary
metrics_all = {}

def dq_df_writeall():
    
    writer = pd.ExcelWriter('dqanalysis.xlsx', mode = 'w')
    
    first = True
    for date, df in dq_dfDict.items():
        df.to_excel(writer, sheet_name=date)
        metrics_all[date + ('' if first else '-new')] = sum_metrics(df)
        first = False
        
    df = pd.DataFrame(metrics_all)
    df.to_excel(writer, sheet_name="DQ Metrics")
    writer.save()

In [ ]:
#for each file that matches a filename pattern, we will load the file, print out the numbers
#as listed based on column mismatches and then we will put the numbers in a comparison array with the date of file 
#as one of the parameters. Then we should be able to establish a trend.
#example file name: participant_info_12_09_2023_01_03_40.xlsx

import os,glob
searchedfiles = sorted(glob.glob("./dbcompare_info_*.xlsx"), key=os.path.getmtime)

for fname in searchedfiles:
    #date extraction
    m = re.match(r'.\/dbcompare_info_(\d\d)_(\d\d)_(\d\d\d\d).*', fname)
    dq_date = m.group(2) + "-" + m.group(1) + "-" + m.group(3)
    print(dq_date)
    
    dq_df = pd.read_excel(fname, 'differences')
    
    #metrics_base = dict.fromkeys(metrics_base, False)
    dq_df = dq_clean(dq_df)
    dq_df = dq_analyze(dq_df, dq_date)
    dq_df_add(dq_df, dq_date)


In [ ]:
dq_df_writeall()